In [36]:
import requests
from bs4 import BeautifulSoup
import re
from lxml import etree
import json
from pygtrans import Translate
import csv

IMDB_MOVIE_FILENAME = "imdb_top_250.json"
DOUBAN_MOVIE_FILENAME = "douban_top_250.json"
DOUBAN_MOVIE_CSV = "douban_top250.csv"

# imdb_url = 'http://www.imdb.com/chart/top'
# imdb_req = requests.get(imdb_url)
# imdb_soup = BeautifulSoup(imdb_req.text, 'lxml')
# imdb_movies = imdb_soup.select('td.titleColumn')





In [110]:
class imdb_movie:
    def __init__(self ,movie_id = None, url = None, name = None, rating = None, duration = None, short_description = None, year = None, genre = None, directors = None, actors = None, ranking = None, review_num = None, top_review_content = None):
        self.url = url
        self.name = name
        self.rating = rating
        self.duration = duration
        self.short_description = short_description
        self.year = year
        self.genre = genre
        self.directors = directors
        self.actors = actors
        self.ranking = ranking
        self.movie_id = movie_id
        self.review_num = review_num
        self.top_review_content = top_review_content
    
    def movie_imdb_info(self):
        data = {
            "url": self.url,
            "movie_id": self.movie_id
        }
        return data
    
    def movie_basic_info(self):
        data = {
            "genre": self.genre,
            "year": self.year,
            "duration": self.duration,
            "actors": self.actors,
            "directors": self.directors}
        return data
    
    def quantitative_info(self):
        data = {
            "ranking": self.ranking,
            "rating": self.rating,
            "review_num": self.review_num,
            "top_review_content": self.top_review_content}
        return data
    
    def movie_info(self):
        data = { "name": self.name, 
                "imdb_info": self.movie_imdb_info(),
                "basic_info": self.movie_basic_info(),
                "quantitative_info": self.quantitative_info()}
        return data


In [111]:
def build_imdb_list_from_top_250():

    imdb_url = 'http://www.imdb.com/chart/top'
    imdb_req = requests.get(imdb_url)
    imdb_soup = BeautifulSoup(imdb_req.text, 'lxml')
    imdb_movies = imdb_soup.select('td.titleColumn')

    imdb_url = ['https://www.imdb.com' + a.attrs.get('href') for a in imdb_soup.select('td.titleColumn a')]
    imdb_actors = [a.attrs.get('title') for a in imdb_soup.select('td.titleColumn a')]
    imdb_ratings = [b.attrs.get('data-value') for b in imdb_soup.select('td.posterColumn span[name=ir]')]

    imdb_list_in = []

    for index in range(0, len(imdb_movies)):
        movie_string = imdb_movies[index].get_text()
        movie = (' '.join(movie_string.split()).replace('.', ''))
        movie_crew_split = imdb_actors[index].split(',')
        movie_dir = movie_crew_split[0].strip('(.dir)')
        movie_actor = re.sub('^[^,]*(?=,),', '', imdb_actors[index])
        movie_title = movie[len(str(index))+1:-7]
        year = re.search('\((.*?)\)', movie_string).group(1)
        ranking = movie[:len(str(index))-(len(movie))]
        movie_ids = imdb_url[index].split('/')[-2]
        imdb_data = imdb_movie(url = imdb_url[index], movie_id = movie_ids, name  = movie_title, rating = imdb_ratings[index], year = year, actors = movie_actor, ranking = ranking, directors=movie_dir)
        imdb_list_in.append(imdb_data)
    return imdb_list_in

imdb_list = build_imdb_list_from_top_250()

In [73]:
sub_url = imdb_list[0].url
sub_req = requests.get(sub_url)
sub_soup = BeautifulSoup(sub_req.text, 'lxml')
sub_time = sub_soup.find('div', class_='sc-94726ce4-3 eSKKHi').find_all("li")[-1].get_text()
sub_rated_num = sub_soup.find('div', class_= 'sc-66a20916-0').select('span')[0].get_text()
sub_top_review_content = sub_soup.find_all('div', class_='ipc-html-content-inner-div')[-1].get_text()
sub_genre = sub_soup.find_all('div', class_='sc-16ede01-4')[-1].get_text()
sub_description = sub_soup.select('span.sc-16ede01-1.kgphFu')[0].get_text()
pattern = "[A-Z]"
sub_genre = re.sub(pattern, lambda x: " " + x.group(0), sub_genre)

print((sub_top_review_content))


If you like hopeful, surprising, never-seen-before characters, you will enjoy this amusing story of a family of prisoners victimized by the system and a Bible thumping pig.Robbins and Freeman, and everybody else, gives perfect performances for their characters. Their actions and body languages are perfect for this story and movie.


In [112]:
def build_imdb_list_from_sub_html():
    imdb_list = build_imdb_list_from_top_250()
    for index in range(0, len(imdb_list)):
        sub_url = imdb_list[index].url
        sub_req = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_req.text, 'lxml')
        sub_time = sub_soup.find('div', class_='sc-94726ce4-3 eSKKHi').find_all("li")[-1].get_text()
        sub_description = sub_soup.select('span.sc-16ede01-1.kgphFu')[0].get_text()
        sub_rated_num = sub_soup.find('div', class_= 'sc-66a20916-0').select('span')[0].get_text()
        sub_top_review_content = sub_soup.find_all('div', class_='ipc-html-content-inner-div')[-1].get_text()
        try:
            init_genre = sub_soup.find_all('div', class_='sc-16ede01-4')[-1].get_text()
        except : IndexError
        pattern = "[A-Z]"
        sub_genre = re.sub(pattern, lambda x: " " + x.group(0), init_genre)
        imdb_list[index].genre = sub_genre
        imdb_list[index].duration = sub_time
        imdb_list[index].short_description = sub_description
        imdb_list[index].review_num = sub_rated_num
        imdb_list[index].top_review_content = sub_top_review_content

    return imdb_list

imdb_list = build_imdb_list_from_sub_html()

In [113]:
# organize the name and duration as basic information, and the rest as detailed information
def save_imdb_cache(imdb_list: imdb_movie):
    f = open(IMDB_MOVIE_FILENAME, 'w').close()
    for index in range(0, len(imdb_list)):
        imdb_data = imdb_movie.movie_info(imdb_list[index])
        jsonStr = json.dumps(imdb_data)
        with open(IMDB_MOVIE_FILENAME, 'a') as f:
            f.write(jsonStr + '\n')
    f.close()

save_imdb_cache(imdb_list)



In [42]:
class douban_movie:
    def __init__(self, name = None, rating = None, duration = None, short_description = None, year = None, genre = None, directors = None, actors = None, ranking = None, rating_num = None, top_review_content = None):
        self.name = name
        self.rating = rating
        self.duration = duration
        self.short_description = short_description
        self.year = year
        self.genre = genre
        self.directors = directors
        self.actors = actors
        self.ranking = ranking
        self.rating_num = rating_num
        self.top_review_content = top_review_content
        
    def movie_basic_info(self):
        data = {
            # "genre": self.genre,
            "year": self.year,
            "duration": self.duration,
            # "actors": self.actors,
            "directors": self.directors}
        return data
    
    def quantitative_info(self):
        data = {
            "ranking": self.ranking,
            "rating": self.rating,
            "rating_num": self.rating_num}
            # "top_review_content": self.top_review_content}
        return data
    
    def movie_info(self):
        data = { "name": self.name,
                "basic_info": self.movie_basic_info(),
                "quantitative_info": self.quantitative_info()}
        return data


In [43]:

# csv_reader = csv.reader(open("douban_top250.csv", encoding='utf-8'))
def build_douban_list(csv_filename):
    douban_list = []
    csv_reader = csv.reader(open(csv_filename, encoding='utf-8'))
    for line in csv_reader:
        translate = Translate()
        movie_director = translate.translate(line[1], target='en').translatedText
        movie_name = translate.translate(line[2], target='en').translatedText
        movie_ranking = line[3]
        movie_rating= line[4]
        try: movie_duration = str(int(line[6])//60) + "h" + " " + str(int(line[6])%60) + "m"
        except: ValueError
        movie_review_num = line[7]
        movie_year = line[-1]
        douban_data = douban_movie(directors = movie_director, name = movie_name, rating = movie_rating, duration = movie_duration, year = movie_year, ranking = movie_ranking, rating_num = movie_review_num)
        douban_list.append(douban_data)
        print(douban_data.directors)
    return douban_list



douban_list = build_douban_list(DOUBAN_MOVIE_CSV)
# data[1][1]
# translate = Translate()
# translate.translate(data[1][1], target='en').translatedText


Frank Darabont
Chen Kaige
Luc Besson
Robert Zemeckis
Roberto Bernini
Miyazaki
Steven Spielberg
James Cameron
Christopher Nolan
Andrew Stanton
Giuseppe Tonadore
Rajkumar Hirani
Lars Hallstrom
Christopher Barrati
Liu Zhenwei
Miyazaki
Francis Ford Coppola
Peter Weir
Victor Fleming
Giuseppe Tonadore
Olivier Nacash
Gabriele Muccino
Hwang Dong Hyuk
Liu Weiqiang
David Fincher
Sidney Lumet
Rob Reiner
Peter Jackson
Christopher Nolan
Ang Lee
Miyazaki
Jiang Wen
William Wheeler
Christopher Nolan
Liu Zhenwei
Zhang Yimou
Guy Ritchie
Peter Docter
Florian Henkel von Donnersmark
Milos Foreman
Louis Seahoyos
Martin Bryce
James McTegg
Miyazaki
Francis Ford Coppola
Ron Howard
Peter Jackson
Peter Jackson
Peter Weir
Shunji Iwai
Billy Wilder
Jean-Pierre Genet
Sergio Leone
Yang Yushuo
Roman Polanski
Ang Lee
David Fincher
Roger Allers
Tetsuya Nakashima
Christopher Nolan
Majid Majidi
Mel Gibson
Tim Burton
Robert Wise
Byron Howard
Quentin Tarantino
Lee Jun-ik
David Fincher
Yojiro Asada
Jonathan Demme
Lily Wachow

In [44]:
def save_douban_cache(douban_list: douban_movie):
    f = open(DOUBAN_MOVIE_FILENAME, 'w').close()
    for index in range(0, len(douban_list)):
        douban_data = douban_movie.movie_info(douban_list[index])
        jsonStr = json.dumps(douban_data)
        with open(DOUBAN_MOVIE_FILENAME, 'a') as f:
            f.write(jsonStr + '\n')
    f.close()

save_douban_cache(douban_list)